<a href="https://colab.research.google.com/github/marcosdogexd/ProgramacionDispositivosMoviles/blob/main/ScrapyDataBasic_OntanedaMarcos_S10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalación de librerías necesarias para trabajar con datos web y tablas
!pip install beautifulsoup4 requests pandas

In [ ]:
# Importar la librería de Google Colab para conectar Google Drive
from google.colab import drive

# Montar Google Drive en la carpeta '/content/drive' para poder acceder y guardar archivos en tu unidad
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Importar librerías necesarias para procesamiento de datos y análisis web
import json                          # Para manejar datos en formato JSON
import requests                      # Para descargar datos desde una página web
import pandas as pd                  # Para trabajar con datos de tablas
from bs4 import BeautifulSoup as bs  # Para analizar el contenido HTML descargado desde la web

In [ ]:
# Función que convierte una lista de filas en un DataFrame de Pandas
def rowsToDataFrame(rows):


    df = pd.DataFrame(rows[1:], columns=rows[0])  # La primera fila (rows[0]) se usa como encabezados de columna, y las filas siguientes (rows[1:]) son los datos
    return df

In [ ]:
# Función que procesa una tabla HTML y extrae sus datos en una lista de filas
def processTableData(tbl):
    rows = []  # Crear una lista vacía para almacenar las filas de la tabla

    # Recorrer cada hijo en el cuerpo de la tabla
    for child in tbl.find('tbody').children:
        try:
            row = []  # Crea una lista vacía para almacenar cada celda de una fila
            # Busca todos los elementos <th> y <td> de cada fila
            for td in child.find_all(['th', 'td']):
                # eliminar saltos de línea y espacios
                item = td.text.replace('\n', '').strip()
                if item:
                    row.append(item)  # Agrega el texto a la lista de la fila
            if row:
                rows.append(row)  # Agregar la fila a la lista de filas de la tabla
        except:
            continue
    return rows

In [ ]:
# Función que convierte el contenido HTML en un formato analizable y busca la tabla
def processDataHTML(html_content):
    # Convertir el contenido HTML en un objeto BeautifulSoup para poderlo analizar
    soup = bs(html_content, 'html.parser')

    # Buscar la primera tabla que tenga la clase 'wikitable'
    tbl = soup.find('table', {'class': 'wikitable'})

    # procesa la tabla usando processTableData; si no, devuelve una lista vacía
    tblRows = processTableData(tbl) if tbl else []
    return tblRows

In [ ]:
# Especificar la URL de la página de Wikipedia que queremos analizar
url = 'https://es.wikipedia.org/wiki/Deportes_electr%C3%B3nicos'

# Descargar el contenido de la página usando requests.get y lo almacena
r = requests.get(url)

In [ ]:
# Comprueba si la descarga de la página fue exitosa (código 200 significa éxito)
if r.status_code == 200:
    # Procesa el contenido HTML de la página
    table = processDataHTML(r.text)

    # Si se encuentran datos en la tabla, los convierte en un DataFrame de Pandas
    if table:
        df = rowsToDataFrame(table)  # Convertir las filas en un DataFrame
        print(df.info())

        # Guardar el DataFrame en un archivo CSV en Google Drive
        df.to_csv('/content/drive/My Drive/esports_estadisticas.csv', index=False)
        print("Archivo CSV guardado exitosamente en Google Drive.")
    else:
        print("No se encontró ninguna tabla en la página.")
else:
    print(f"Error al acceder a la página: {r.status_code}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Año         17 non-null     object
 1   Torneo      17 non-null     object
 2   Videojuego  17 non-null     object
 3   Campeón     17 non-null     object
 4   Subcampeón  17 non-null     object
dtypes: object(5)
memory usage: 808.0+ bytes
None
Archivo CSV guardado exitosamente en Google Drive.
